In [1]:
import os
import re
import pandas as pd
import math
import sklearn.metrics

In [2]:
# global settings

height = 7
width = 1048576
width_2 = int(width * 1.25)
kmer_len = 22
number_of_rounds = 4

input_fasta_files = [
    "input/lhg22L20MC5x.fa",
    "input/lhg22L20MC10x.fa",
    "input/lhg22L20MC20x.fa",
    "input/lhg22L20MC30x.fa",
    "input/lhg22L20MC50x.fa",
]
input_query_files = [
    "input/test_exact_count_lhg22L20MC5x_20000.txt",
    "input/test_exact_count_lhg22L20MC10x_20000.txt",
    "input/test_exact_count_lhg22L20MC20x_20000.txt",
    "input/test_exact_count_lhg22L20MC30x_20000.txt",
    "input/test_exact_count_lhg22L20MC50x_20000.txt",
]

conservative = False

In [3]:
def get_analysis(output_file: str = "output/query_result.csv"):
    df = pd.read_csv(output_file)
    temp_df = df.loc[df["true_count"] > df["estimated_count"]]
    assert temp_df.empty, "There exists some rows for which truecount > estimated count"
    sp_corr = (
        df[["true_count", "estimated_count"]]
        .corr(method="spearman")["true_count"]
        .drop("true_count")
    )
    rms_error = math.sqrt(
        sklearn.metrics.mean_squared_error(df["true_count"], df["estimated_count"])
    )
    return rms_error, sp_corr.iloc[0]

In [4]:
print(
    "fasta_file,method,rms_error,corr_coef,time_taken_sec,query_file,height,width,number_of_round"
)
for i in range(len(input_fasta_files)):
    # first run the cm
    os.path.exists("cm") and os.system("rm cm")
    output = os.popen("g++ --std=c++17 -o cm cm.cpp").read()
    command = f"./cm count -k {kmer_len} -fa {input_fasta_files[i]} -o sketch_file.sketch -h {height} -w {width}"

    output = os.popen(command).read()
    time_taken_match = re.search(r"Time difference = (\d+)\[sec\]", output)
    time_taken = int(time_taken_match.group(1)) if time_taken_match else None

    # then run the query
    command = f"./cm test -f sketch_file.sketch -q {input_query_files[i]} -o output/query_result.csv"
    output = os.popen(command).read()

    rms_error, sp_corr = get_analysis()

    print(
        f"{input_fasta_files[i]},cm,{rms_error},{sp_corr},{time_taken},{input_query_files[i]},{height},{width},{None}"
    )

    # run the ocm
    os.path.exists("ocm") and os.system("rm ocm")
    output = os.popen("g++ --std=c++17 -o ocm ocm.cpp").read()
    command = f"./ocm count -k {kmer_len} -fa {input_fasta_files[i]} -o sketch_file.sketch -h {height} -w {width} -n {number_of_rounds}"


    output = os.popen(command).read()
    time_taken_match = re.search(r"Time difference = (\d+)\[sec\]", output)
    time_taken = int(time_taken_match.group(1)) if time_taken_match else None

    # then run the query
    command = f"./ocm test -f sketch_file.sketch -q {input_query_files[i]} -o output/query_result.csv"
    output = os.popen(command).read()

    rms_error, sp_corr = get_analysis()

    print(
        f"{input_fasta_files[i]},ocm,{rms_error},{sp_corr},{time_taken},{input_query_files[i]},{height},{width},{number_of_rounds}"
    )

    # run the cm with width 1.25 times
    os.path.exists("cm") and os.system("rm cm")
    output = os.popen("g++ --std=c++17 -o cm cm.cpp").read()
    command = f"./cm count -k {kmer_len} -fa {input_fasta_files[i]} -o sketch_file.sketch -h {height} -w {width_2}"

    output = os.popen(command).read()
    time_taken_match = re.search(r"Time difference = (\d+)\[sec\]", output)
    time_taken = int(time_taken_match.group(1)) if time_taken_match else None

    # then run the query
    command = f"./cm test -f sketch_file.sketch -q {input_query_files[i]} -o output/query_result.csv"
    output = os.popen(command).read()

    rms_error, sp_corr = get_analysis()

    print(
        f"{input_fasta_files[i]},cm2,{rms_error},{sp_corr},{time_taken},{input_query_files[i]},{height},{width_2},{None}"
    )

    # first run the ccm
    os.path.exists("cm") and os.system("rm cm")
    output = os.popen("g++ --std=c++17 -o cm cm.cpp").read()
    command = f"./cm count -c -k {kmer_len} -fa {input_fasta_files[i]} -o sketch_file.sketch -h {height} -w {width}"

    output = os.popen(command).read()
    time_taken_match = re.search(r"Time difference = (\d+)\[sec\]", output)
    time_taken = int(time_taken_match.group(1)) if time_taken_match else None

    # then run the query
    command = f"./cm test -f sketch_file.sketch -q {input_query_files[i]} -o output/query_result.csv"
    output = os.popen(command).read()

    rms_error, sp_corr = get_analysis()

    print(
        f"{input_fasta_files[i]},ccm,{rms_error},{sp_corr},{time_taken},{input_query_files[i]},{height},{width},{None}"
    )

    # run the occm
    os.path.exists("ocm") and os.system("rm ocm")
    output = os.popen("g++ --std=c++17 -o ocm ocm.cpp").read()
    command = f"./ocm count -c -k {kmer_len} -fa {input_fasta_files[i]} -o sketch_file.sketch -h {height} -w {width} -n {number_of_rounds}"


    output = os.popen(command).read()
    time_taken_match = re.search(r"Time difference = (\d+)\[sec\]", output)
    time_taken = int(time_taken_match.group(1)) if time_taken_match else None

    # then run the query
    command = f"./ocm test -f sketch_file.sketch -q {input_query_files[i]} -o output/query_result.csv"
    output = os.popen(command).read()

    rms_error, sp_corr = get_analysis()

    print(
        f"{input_fasta_files[i]},occm,{rms_error},{sp_corr},{time_taken},{input_query_files[i]},{height},{width},{number_of_rounds}"
    )

    # run the ccm with width 1.25 times
    os.path.exists("cm") and os.system("rm cm")
    output = os.popen("g++ --std=c++17 -o cm cm.cpp").read()
    command = f"./cm count -c -k {kmer_len} -fa {input_fasta_files[i]} -o sketch_file.sketch -h {height} -w {width_2}"

    output = os.popen(command).read()
    time_taken_match = re.search(r"Time difference = (\d+)\[sec\]", output)
    time_taken = int(time_taken_match.group(1)) if time_taken_match else None

    # then run the query
    command = f"./cm test -f sketch_file.sketch -q {input_query_files[i]} -o output/query_result.csv"
    output = os.popen(command).read()

    rms_error, sp_corr = get_analysis()

    print(
        f"{input_fasta_files[i]},ccm2,{rms_error},{sp_corr},{time_taken},{input_query_files[i]},{height},{width_2},{None}"
    )

fasta_file,method,rms_error,corr_coef,time_taken_sec,query_file,height,width,number_of_round
input/lhg22L20MC5x.fa,cm,25.943468718627237,0.7217882359241325,27,input/test_exact_count_lhg22L20MC5x_20000.txt,7,1048576,None
input/lhg22L20MC5x.fa,ocm,14.563449406116003,0.7405669933897289,445,input/test_exact_count_lhg22L20MC5x_20000.txt,7,1048576,4
input/lhg22L20MC5x.fa,cm2,19.181045981858713,0.7460603859927465,27,input/test_exact_count_lhg22L20MC5x_20000.txt,7,1310720,None
input/lhg22L20MC5x.fa,ccm,8.737287148431982,0.7579151832382,32,input/test_exact_count_lhg22L20MC5x_20000.txt,7,1048576,None
input/lhg22L20MC5x.fa,occm,8.708737131350729,0.7578454395418199,397,input/test_exact_count_lhg22L20MC5x_20000.txt,7,1048576,4
input/lhg22L20MC5x.fa,ccm2,6.043704264152018,0.7980849575033607,32,input/test_exact_count_lhg22L20MC5x_20000.txt,7,1310720,None
input/lhg22L20MC10x.fa,cm,29.083246153038996,0.7213255068389548,29,input/test_exact_count_lhg22L20MC10x_20000.txt,7,1048576,None
input/lhg22L20MC10x